In [27]:
import polib
from openai import OpenAI
from dotenv import load_dotenv
import os
from tqdm import tqdm
import glob
%load_ext autoreload
%autoreload 2
from common import desired_text, add_space_around_special_phrase, translate_text_to_japanese, create_batches

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [29]:
files = glob.glob("../locales/en/LC_MESSAGES/**/*.po", recursive=True)
files.sort()

In [30]:
for file in tqdm(files, desc="外側のループ"):

    opath = file.replace("/en/", "/ja/")

    if os.path.exists(opath):
        continue

    with open(file, "r") as f:
        po_text = f.read()

    print(file, len(po_text))

    lines = po_text.split("\n\n")

    batches = create_batches(lines)

    result_merged = ""

    for batch in tqdm(batches, desc="内側のループ", leave=False):

        po_batch_text = "\n\n".join(batch)

        po_batch_text = f'''# SOME DESCRIPTIVE TITLE.
# Copyright (C) 2023, Artefactual Systems Inc.
# This file is distributed under the same license as the Archivematica
# package.
# FIRST AUTHOR <EMAIL@ADDRESS>, 2024.
#
#, fuzzy
msgid ""
msgstr ""

### START ###

{po_batch_text}'''

        result = translate_text_to_japanese(client, po_batch_text)

        fixed_result = desired_text(result)

        result_merged += fixed_result + "\n\n"
        
    opath = file.replace("/en/", "/ja/")

    os.makedirs(os.path.dirname(opath), exist_ok=True)

    with open(opath, "w") as f:
        f.write(result_merged)

    try:
        po = polib.pofile(opath)

        for entry in po:
            msgstr = entry.msgstr
            entry.msgstr = add_space_around_special_phrase(msgstr)

        
        os.makedirs(os.path.dirname(opath), exist_ok=True)

        po.save(opath)

    except Exception as e:
        print(e)
        print(opath)
        continue

外側のループ:   0%|          | 0/8 [00:00<?, ?it/s]

../locales/en/LC_MESSAGES/administrators.po 65510


外側のループ:  12%|█▎        | 1/8 [39:17<4:35:03, 2357.59s/it]

../locales/en/LC_MESSAGES/contents.po 979


外側のループ:  25%|██▌       | 2/8 [39:32<1:37:57, 979.58s/it] 

...

外側のループ:  75%|███████▌  | 6/8 [46:24<07:18, 219.38s/it]

../locales/en/LC_MESSAGES/migration-sqlite-mysql.po 2302


外側のループ:  88%|████████▊ | 7/8 [47:10<02:42, 162.63s/it]

Syntax error in po file ../locales/ja/LC_MESSAGES/migration-sqlite-mysql.po (line 79)
../locales/ja/LC_MESSAGES/migration-sqlite-mysql.po
../locales/en/LC_MESSAGES/recovery.po 5959


外側のループ: 100%|██████████| 8/8 [49:35<00:00, 371.92s/it]
